# Coverage tests for collected tweets

Estimate what percentage of Dutch tweets are collected by twiqs.nl

In [3]:
import math
import os
import pandas as pd
import re
import sys
from IPython.display import clear_output

In [4]:
DATADIR = "/home/erikt/projects/puregome/data/text/"

In [5]:
def squeal(text=None):
    clear_output(wait=True)
    if not text is None: print(text)

In [ ]:
IDSTR = "id_str"
INREPLYTOSTATUSIDSTR = "in_reply_to_status_id_str"

def getReplyPercentage(filePattern,filePatternReference):
    files = sorted(os.listdir(DATADIR))
    targetFiles = {}
    counter = 0
    for inFileName in files:
        if re.search(filePattern,inFileName):
            counter += 1
            df = pd.read_csv(DATADIR+inFileName,dtype={INREPLYTOSTATUSIDSTR:object})
            counter += len(df)
            for idStr in df[INREPLYTOSTATUSIDSTR]:
                if type(idStr) == type("abc"):
                    targetFiles[idStr] = False
    for inFileName in files:
        if re.search(filePatternReference,inFileName):
            df = pd.read_csv(DATADIR+inFileName,dtype={IDSTR:object})
            for idStr in df[IDSTR]: 
                if idStr in targetFiles: targetFiles[idStr] = True
            squeal(inFileName)
    squeal("")
    return(len([x for x in targetFiles if targetFiles[x]]),len(targetFiles),counter)

In [ ]:
filePattern = "20200609"
filePatternReference = "202006"
lenPart,lenAll,counter = getReplyPercentage(filePattern,filePatternReference)
print("target date: {0}; reference: {1}; percentage: {2}%; total count: {3}".format(filePattern,filePatternReference,\
                                                                                   round(100*lenPart/lenAll,1),counter))

In [ ]:
filePattern = "20200610"
lenPart,lenAll,counter = getReplyPercentage(filePattern,filePatternReference)
print("target date: {0}; reference: {1}; percentage: {2}%; total count: {3}".format(filePattern,filePatternReference,\
                                                                                   round(100*lenPart/lenAll,1),counter))

In [ ]:
filePattern = "20200611"
lenPart,lenAll,counter = getReplyPercentage(filePattern,filePatternReference)
print("target date: {0}; reference: {1}; percentage: {2}%; total count: {3}".format(filePattern,filePatternReference,\
                                                                                   round(100*lenPart/lenAll,1),counter))

## Find best keywords

In [1]:
from nltk.tokenize import TweetTokenizer

In [41]:
FILEPATTERN = "202006"
TEXT = "text"
BASELEXICON = set(topTokens100 + topTokens200 + topTokens300 + topTokens400)
ALLCOUNT = 596893 # 5623571
LEFT200 = 61599
LEFT100 = 80258 # 349598
LEFT200100 = 43622 # 179089
LEFT300200100 = 34583 # 130887
LEFT400300200100 = 29528 # 109099

def getTweets():
    files = sorted(os.listdir(DATADIR))
    tweets = {}
    for inFileName in files:
        if re.search(FILEPATTERN,inFileName):
            squeal(inFileName)
            df = pd.read_csv(DATADIR+inFileName)
            for text in df[TEXT]:
                text = re.sub(r"\\n"," ",text.lower())
                text = re.sub(r"[#@]"," ",text)
                tweets[text] = True
    return(tweets)

# tweets = getTweets()

tokenizer = TweetTokenizer()
frequencies = {}
processedTweetCount = 0
toBeDeleted = []
counter = 0
for text in tweets.keys():
    counter += 1
    if counter%10000 == 0: squeal(str(counter)+"/"+str(len(tweets)))
    tokens = set(tokenizer.tokenize(text.lower()))
    if len(BASELEXICON.intersection(tokens)) > 0:
        toBeDeleted.append(text)
    else:
        processedTweetCount += 1
        for token in tokens:
            if token in frequencies: frequencies[token] += 1
            else: frequencies[token] = 1
for text in toBeDeleted: del(tweets[text])
squeal()
processedTweetCount

109099

In [50]:
for token in sorted(BASELEXICON): print(token,end=",")
print()

0,0000,1,10,11,12,18,19,2,20,2020,2020-06-,3,4,5,6,7,71,8,9,:(,:),<3,a,a1,a2,aan,af,al,alle,alleen,allemaal,alles,als,altijd,alweer,ameen,amsterdam,an,and,b,baby,bark,bedankt,beetje,beke,ben,best,beter,beterschap,biep,bij,bijna,black,blonde,boep,buiten,by,c,code,corona,da,daar,dag,dan,daniel,dank,dankje,dankjewel,dat,de,den,deze,die,dit,do,doe,doen,doet,door,dus,e,echt,ee,een,eens,eerst,eerste,eindelijk,en,er,erg,es,et,even,ewdevlieger,ff,fijn,fijne,for,foto,ga,gaan,gaat,ge,gecondoleerd,gedaan,geen,geertwilderspvv,gefeliciteerd,gelijk,gelukkig,geniet,geweldig,gewoon,gezellig,gij,goed,goede,goedemorgen,goeie,goeiemorgen,graag,gwn,h,haar,haattweetje,had,haha,hahaha,hahahah,hahahaha,hahahahah,hahahahaha,hahahahahaha,halsema,he,heb,hebben,heeft,heel,heerlijk,hele,helemaal,help,herkenbaar,het,hi,hier,hij,hoe,hoezo,hoop,hoor,hun,hè,i,ie,iedereen,iemand,iets,ig,ik,im,in,inderdaad,ingeborgvraagt,is,ja,jaa,jaar,jazeker,je,jij,jndkgrf,joonie,joostniemoller,jou,jullie,k,ka,kan,kangen,ke,kijk,kijk

In [43]:
sortedFrequencies = {k:frequencies[k] for k in sorted(frequencies.keys(),key=lambda k:frequencies[k],reverse=True)}
topTokens500 = [k for k in sortedFrequencies.keys() if re.search(r"[a-z0-9]",k) or \
                                                       (len(k) > 1 and k != ".." and k != "..." or
                                                       (len(k) == 1 and ord(k) >= 127000 and ord(k) < 130000))][0:100]